### what issues are there in the first 1000 rows?

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType
#from pyspark.sql.functions import to_date, date_format, col, when
#from datetime import datetime


# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read JSON with PySpark") \
    .getOrCreate()

In [2]:
# Read JSON file into DataFrame
# plit out related field to get buy_after_viewing

meta_schema = StructType([
    StructField("asin", StringType(), True),
   StructField("categories", ArrayType(ArrayType(StringType()),True), True) 

    
])


metadata = spark.read.json("metadata.json", schema=meta_schema)

In [3]:
# Display DataFrame schema
metadata.printSchema()
print((metadata.count(), len(metadata.columns)))

root
 |-- asin: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)

(9430088, 2)


In [4]:

from pyspark.sql.functions import explode,flatten
exploded = metadata.select(metadata.asin,explode(metadata.categories))

In [5]:
exploded = exploded.select(exploded.asin,explode(exploded.col).alias('category'))

In [6]:
print((exploded.count(), len(exploded.columns)))

(44701384, 2)


In [7]:
exploded = exploded.repartition(20)
exploded.write.mode("overwrite").json("metadata_splits")